# TOC Categorization Demo

**Purpose:** Demonstrate the 13-category system for classifying TOC entries by section type.

**Key Features:**
- 13 semantic categories covering all well report sections
- Fuzzy matching for flexible categorization
- Coverage improvement: 62.8% → 98.5%
- Section type filtering for improved RAG retrieval

**Categories:**
1. project_admin - Project info, permits, dates
2. well_identification - Well name, location, coordinates
3. geology - Formation tops, lithology, stratigraphy
4. borehole - Depths, trajectory, hole sizes
5. casing - Casing program, completion strings
6. directional - Directional drilling, surveys
7. drilling_operations - Drilling parameters, fluids, BHA
8. completion - Perforation, gravel pack, stimulation
9. technical_summary - Summaries, conclusions, recommendations
10. hse - Health, safety, environment
11. appendices - Supplementary materials, corrupted entries
12. well_testing - Production tests, pressure tests, FIT
13. intervention - Workover, perforating, TCP operations

**Achievement:** 204/207 entries categorized (98.5%)

In [ ]:
# Setup environment
import sys
from pathlib import Path
import os
import json

# Navigate to project root
project_root = Path.cwd().parent.parent
os.chdir(project_root)
sys.path.insert(0, str(project_root / "src"))
sys.path.insert(0, str(project_root / "scripts"))

print(f"Project root: {project_root}")
print("[OK] Environment setup complete")

## Load Categorization Data

Load the 13-category mapping and TOC database.

In [ ]:
# Load 13-category mapping
categorization_path = project_root / "outputs" / "final_section_categorization_v2.json"

if categorization_path.exists():
    with open(categorization_path, 'r') as f:
        categorization = json.load(f)
    
    print(f"[OK] Loaded categorization: version {categorization['metadata']['version']}")
    print(f"[OK] Total categories: {categorization['metadata']['total_categories']}")
    
    # Count total entries
    total_entries = sum(len(cat['entries']) for cat in categorization['categories'].values())
    print(f"[OK] Total categorized entries: {total_entries}")
else:
    print(f"[ERROR] Categorization file not found at {categorization_path}")
    print("Run scripts/create_improved_categorization.py to generate it")

In [ ]:
# Load TOC database
toc_database_path = project_root / "outputs" / "exploration" / "toc_database_multi_doc_full.json"

if toc_database_path.exists():
    with open(toc_database_path, 'r') as f:
        toc_database = json.load(f)
    
    # Count total TOC entries
    total_toc_entries = sum(
        len(doc['toc']) 
        for well_docs in toc_database.values() 
        for doc in well_docs
    )
    
    print(f"[OK] Loaded TOC database")
    print(f"[OK] Wells: {len(toc_database)}")
    print(f"[OK] Total TOC entries: {total_toc_entries}")
else:
    print(f"[ERROR] TOC database not found at {toc_database_path}")
    print("Run scripts/build_multi_doc_toc_database_full.py to generate it")

## Category Distribution

Visualize how TOC entries are distributed across the 13 categories.

In [ ]:
print("="*120)
print("CATEGORY DISTRIBUTION")
print("="*120)
print(f"{'Category':<25} {'Entries':<10} {'%':<8} {'Description'}")
print("-"*120)

# Sort by entry count (descending)
sorted_categories = sorted(
    categorization['categories'].items(),
    key=lambda x: len(x[1]['entries']),
    reverse=True
)

total_entries = sum(len(cat['entries']) for _, cat in categorization['categories'].items())

for cat_name, cat_data in sorted_categories:
    entry_count = len(cat_data['entries'])
    percentage = entry_count / total_entries * 100 if total_entries > 0 else 0
    
    # Create bar chart
    bar_length = int(percentage / 2)  # Scale to 50 chars max
    bar = '█' * bar_length
    
    print(f"{cat_name:<25} {entry_count:<10} {percentage:5.1f}%   {bar}")
    print(f"{'':25} {cat_data['description']}")
    print()

print("-"*120)
print(f"{'TOTAL':<25} {total_entries:<10} 100.0%")
print("="*120)

## Coverage Analysis

Compare categorization coverage before and after the 13-category system.

In [ ]:
print("="*100)
print("COVERAGE IMPROVEMENT ANALYSIS")
print("="*100)

# Count categorized entries in TOC database
categorized_count = 0
uncategorized_count = 0
total_count = 0

for well_name, documents in toc_database.items():
    for doc in documents:
        for entry in doc.get('toc', []):
            total_count += 1
            if entry.get('type'):
                categorized_count += 1
            else:
                uncategorized_count += 1

coverage_percentage = categorized_count / total_count * 100 if total_count > 0 else 0

print(f"\nCurrent Coverage:")
print(f"  Categorized:   {categorized_count:3d} entries ({coverage_percentage:5.1f}%)")
print(f"  Uncategorized: {uncategorized_count:3d} entries ({(100-coverage_percentage):5.1f}%)")
print(f"  Total:         {total_count:3d} entries")

# Historical data (before 13-category system)
old_coverage = 62.8  # From 11-category system
old_categorized = int(total_count * old_coverage / 100)
old_uncategorized = total_count - old_categorized

print(f"\nPrevious Coverage (11-category system):")
print(f"  Categorized:   {old_categorized:3d} entries ({old_coverage:5.1f}%)")
print(f"  Uncategorized: {old_uncategorized:3d} entries ({(100-old_coverage):5.1f}%)")

# Improvement
improvement = coverage_percentage - old_coverage
additional_categorized = categorized_count - old_categorized

print(f"\nImprovement:")
print(f"  Coverage increase:     +{improvement:5.1f}%")
print(f"  Additional entries:    +{additional_categorized:3d} entries")
print(f"  New categories added:  2 (well_testing, intervention)")

print("\n" + "="*100)

## Category Examples

Show example entries for each category.

In [ ]:
print("="*120)
print("CATEGORY EXAMPLES")
print("="*120)

for cat_name, cat_data in categorization['categories'].items():
    entries = cat_data['entries']
    
    if not entries:
        continue
    
    print(f"\n{cat_name.upper()}")
    print(f"Description: {cat_data['description']}")
    print(f"Total entries: {len(entries)}")
    print("-"*120)
    print(f"{'Well':<12} {'Number':<10} {'Title'}")
    print("-"*120)
    
    # Show first 5 examples
    for entry in entries[:5]:
        well = entry['well']
        number = entry['number']
        title = entry['title'][:70]  # Truncate long titles
        print(f"{well:<12} {number:<10} {title}")
    
    if len(entries) > 5:
        print(f"{'':12} {'':10} ... and {len(entries)-5} more")

print("\n" + "="*120)

## New Categories Spotlight

Highlight the 2 new categories added in v2: `well_testing` and `intervention`.

In [ ]:
print("="*120)
print("NEW CATEGORIES (v2)")
print("="*120)

new_categories = ['well_testing', 'intervention']

for cat_name in new_categories:
    if cat_name in categorization['categories']:
        cat_data = categorization['categories'][cat_name]
        entries = cat_data['entries']
        
        print(f"\n{cat_name.upper()}")
        print(f"Description: {cat_data['description']}")
        print(f"Total entries: {len(entries)}")
        print("-"*120)
        
        # Show all entries (not many)
        for i, entry in enumerate(entries, 1):
            well = entry['well']
            number = entry['number']
            title = entry['title']
            print(f"{i:2d}. {well:<12} | {number:<10} | {title}")
        
        print(f"\nWhy this category was added:")
        if cat_name == 'well_testing':
            print("  - Many entries related to production tests, pressure tests, FIT")
            print("  - Previously uncategorized or mixed with 'completion'")
            print("  - Important for understanding well performance")
        elif cat_name == 'intervention':
            print("  - Entries related to workover operations, perforating, TCP")
            print("  - Previously uncategorized or mixed with 'completion'")
            print("  - Distinct from initial completion activities")

print("\n" + "="*120)

## Uncategorized Entries

Show remaining uncategorized entries (should be very few).

In [ ]:
print("="*120)
print("UNCATEGORIZED ENTRIES")
print("="*120)

uncategorized_entries = []

for well_name, documents in toc_database.items():
    for doc in documents:
        for entry in doc.get('toc', []):
            if not entry.get('type'):
                uncategorized_entries.append({
                    'well': well_name,
                    'filename': doc['filename'],
                    'number': entry['number'],
                    'title': entry['title'],
                    'page': entry.get('page', 'N/A')
                })

if uncategorized_entries:
    print(f"\nFound {len(uncategorized_entries)} uncategorized entries:\n")
    print(f"{'Well':<12} {'Number':<10} {'Page':<6} {'Title'}")
    print("-"*120)
    
    for entry in uncategorized_entries:
        well = entry['well']
        number = entry['number']
        page = str(entry['page'])
        title = entry['title'][:70]
        print(f"{well:<12} {number:<10} {page:<6} {title}")
    
    print(f"\nThese entries are likely:")
    print("  - Corrupted/malformed TOC entries")
    print("  - Non-standard section titles")
    print("  - Entries that should be in 'appendices' category")
else:
    print("\n[OK] No uncategorized entries found! (100% coverage)")

print("\n" + "="*120)

## Category Lookup Function

Demonstrate how to find the category for a TOC entry.

In [ ]:
from typing import Optional

def find_category(well: str, section_number: str, section_title: str) -> Optional[str]:
    """
    Find category for a TOC entry using fuzzy matching
    
    Args:
        well: Well name (e.g., "Well 1")
        section_number: Section number (e.g., "2.1")
        section_title: Section title (e.g., "Depths")
    
    Returns:
        Category name or None if not found
    """
    # Search all categories
    for cat_name, cat_data in categorization['categories'].items():
        for entry in cat_data['entries']:
            # Exact match on well + number + title
            if (entry['well'] == well and 
                entry['number'] == section_number and 
                entry['title'].lower() == section_title.lower()):
                return cat_name
            
            # Fuzzy match on title if well and number match
            if (entry['well'] == well and 
                entry['number'] == section_number and 
                (entry['title'].lower() in section_title.lower() or 
                 section_title.lower() in entry['title'].lower())):
                return cat_name
    
    return None

# Test the function
print("="*100)
print("CATEGORY LOOKUP DEMONSTRATION")
print("="*100)

test_cases = [
    ("Well 1", "2.1", "Depths"),
    ("Well 5", "3.2", "Casing"),
    ("Well 7", "4.1", "Drilling fluid"),
    ("Well 2", "1.5", "Production test"),
    ("Well 8", "6.1", "TCP toolstring"),
]

print(f"\n{'Well':<12} {'Section':<10} {'Title':<30} {'Category'}")
print("-"*100)

for well, number, title in test_cases:
    category = find_category(well, number, title)
    if category:
        print(f"{well:<12} {number:<10} {title:<30} {category}")
    else:
        print(f"{well:<12} {number:<10} {title:<30} UNCATEGORIZED")

print("\n[OK] Category lookup working")

## RAG System Integration

Show how section type filtering improves RAG retrieval.

In [ ]:
print("="*100)
print("SECTION TYPE FILTERING FOR RAG")
print("="*100)

# Example queries and recommended filters
query_examples = [
    {
        "query": "What are the measured depths?",
        "filter": "borehole",
        "reason": "Depth information is in borehole sections"
    },
    {
        "query": "Describe the casing program",
        "filter": "casing",
        "reason": "Casing details are in casing sections"
    },
    {
        "query": "What was the drilling fluid composition?",
        "filter": "drilling_operations",
        "reason": "Drilling fluids are in drilling operations sections"
    },
    {
        "query": "What formation tops were encountered?",
        "filter": "geology",
        "reason": "Formation information is in geology sections"
    },
    {
        "query": "What were the production test results?",
        "filter": "well_testing",
        "reason": "Production tests are in well_testing sections"
    },
    {
        "query": "What perforating operations were performed?",
        "filter": "intervention",
        "reason": "Perforating is in intervention sections"
    },
]

print(f"\n{'Query':<50} {'Filter':<20} {'Reason'}")
print("-"*100)

for example in query_examples:
    query = example['query'][:48]
    filter_cat = example['filter']
    reason = example['reason']
    print(f"{query:<50} {filter_cat:<20} {reason}")

print(f"\n\nBenefit of Section Type Filtering:")
print("  1. Improved retrieval accuracy (only relevant sections)")
print("  2. Faster query execution (fewer chunks to search)")
print("  3. More focused answers (no irrelevant context)")
print("  4. Better citation (exact section type in metadata)")

# Count chunks per category (estimate)
print(f"\n\nEstimated Chunk Distribution (1000 char chunks):")
print("-"*100)

for cat_name, cat_data in sorted_categories[:5]:  # Top 5
    entry_count = len(cat_data['entries'])
    estimated_chunks = entry_count * 3  # Rough estimate: 3 chunks per section
    print(f"{cat_name:<25} {entry_count:3d} sections → ~{estimated_chunks:4d} chunks")

print("\n[OK] Section type filtering demonstration complete")

## Per-Well Category Distribution

Analyze category distribution for each well.

In [ ]:
print("="*100)
print("PER-WELL CATEGORY DISTRIBUTION")
print("="*100)

# Collect per-well statistics
well_stats = {}

for well_name, documents in toc_database.items():
    if well_name not in well_stats:
        well_stats[well_name] = {
            'total': 0,
            'categorized': 0,
            'categories': {}
        }
    
    for doc in documents:
        for entry in doc.get('toc', []):
            well_stats[well_name]['total'] += 1
            
            cat_type = entry.get('type')
            if cat_type:
                well_stats[well_name]['categorized'] += 1
                if cat_type not in well_stats[well_name]['categories']:
                    well_stats[well_name]['categories'][cat_type] = 0
                well_stats[well_name]['categories'][cat_type] += 1

# Display statistics
print(f"\n{'Well':<12} {'Total':<8} {'Cat.':<8} {'Coverage':<10} {'Top 3 Categories'}")
print("-"*100)

for well_name in sorted(well_stats.keys()):
    stats = well_stats[well_name]
    total = stats['total']
    categorized = stats['categorized']
    coverage = categorized / total * 100 if total > 0 else 0
    
    # Get top 3 categories
    top_cats = sorted(
        stats['categories'].items(),
        key=lambda x: x[1],
        reverse=True
    )[:3]
    
    top_cats_str = ", ".join(f"{cat}({count})" for cat, count in top_cats)
    
    print(f"{well_name:<12} {total:<8} {categorized:<8} {coverage:5.1f}%     {top_cats_str}")

print("\n" + "="*100)

## Summary

**Achievement:** 204/207 entries categorized (98.5%)

**13-Category System Benefits:**
1. Comprehensive coverage of all well report sections
2. Semantic organization for better understanding
3. Section type filtering for improved RAG retrieval
4. Standardized vocabulary across different wells
5. Easy to extend with new categories

**Improvement Over 11-Category System:**
- Coverage: 62.8% → 98.5% (+35.7%)
- New categories: +2 (well_testing, intervention)
- Additional categorized entries: +76 entries

**Key Categories for Well Analysis:**
- **borehole** - Depths, trajectory (critical for nodal analysis)
- **casing** - Completion strings (critical for nodal analysis)
- **drilling_operations** - Drilling fluids, BHA
- **geology** - Formation tops, lithology
- **well_testing** - Production tests, pressure tests

**RAG System Integration:**
- Filter queries by section type for better accuracy
- Example: "What are the depths?" → filter="borehole"
- Reduces search space and improves answer quality

**Implementation:**
- Categories defined in `outputs/final_section_categorization_v2.json`
- Fuzzy matching for flexible categorization
- Used in `src/chunker.py` for metadata enrichment